In [8]:
import json
file_path = "/mnt/pfs-guan-ssai/nlu/zhangkehao/Unified_Memory_Agent/data/processed_longmemeval.json"
data = json.load(open(file_path))

In [13]:
data[2]['questions'][0]

{'qid': 'longmemeval_51a45a95_0',
 'query': '[2023/05/30 (Tue) 20:42] Where did I redeem a $5 coupon on coffee creamer?',
 'answer': 'Target',
 'position': 43,
 'category': 'single-session-user'}